In [16]:
import pandas as pd
import numpy as np
import panel as pn
import hvplot.pandas
from IPython.display import display, HTML

pn.extension('tabulator')
pd.set_option('display.max_columns', None)

In [17]:
if 'data' not in pn.state.cache.keys():
    animes = pd.read_csv('./anime_data_2006_2022_cleaned.csv')
    pn.state.cache['data'] = animes.copy()
else: 
    animes = pn.state.cache['data']

In [18]:
animes.head()

,mal_id,url,title,title_english,title_japanese,title_synonyms,type,source,episodes,status,airing,duration,rating,score,scored_by,rank,popularity,members,favorites,synopsis,background,year,season,producers,licensors,studios,genres,themes,demographics,image_url,trailer_url,aired_date,broadcast_day_and_time
0,853,https://myanimelist.net/anime/853/Ouran_Koukou...,Ouran Koukou Host Club,Ouran High School Host Club,桜蘭高校ホスト部,"Ohran Koko Host Club,Ouran Koukou Hosutobu,Our...",TV,Manga,26.0,Finished Airing,False,23 min per ep,PG-13 - Teens 13 or older,8.16,672279.0,413.0,128,1110503,34094,Haruhi Fujioka is a studious girl who has rece...,In addition to this anime and the source manga...,2006,spring,"VAP,Hakusensha,Nippon Television Network",Funimation,Bones,"Comedy,Romance","Crossdressing,Reverse Harem,School",Shoujo,https://cdn.myanimelist.net/images/anime/2/719...,https://www.youtube.com/watch?v=NcC5VCE2Its,"Apr 5, 2006 to Sep 27, 2006",Wednesdays at 00:50 (JST)
1,918,https://myanimelist.net/anime/918/Gintama,Gintama,Gintama,銀魂,"Gin Tama,Silver Soul,Yorinuki Gintama-san",TV,Manga,201.0,Finished Airing,False,24 min per ep,PG-13 - Teens 13 or older,8.94,391827.0,15.0,139,1052162,58905,Edo is a city that was home to the vigor and a...,Several games based on Gintama have been relea...,2006,spring,"TV Tokyo,Aniplex,Dentsu,Trinity Sound,Audio Hi...","Sentai Filmworks,Crunchyroll",Sunrise,"Action,Comedy,Sci-Fi","Gag Humor,Historical,Parody,Samurai",Shounen,https://cdn.myanimelist.net/images/anime/10/73...,NaN,"Apr 4, 2006 to Mar 25, 2010",Thursdays at 18:00 (JST)
2,889,https://myanimelist.net/anime/889/Black_Lagoon,Black Lagoon,Black Lagoon,BLACK LAGOON,NaN,TV,Manga,12.0,Finished Airing,False,23 min per ep,R - 17+ (violence & profanity),8.03,487522.0,573.0,155,993437,18516,Salaryman Rokurou Okajima spends his days tryi...,Black Lagoon was released on DVD by Geneon Ent...,2006,spring,"Geneon Universal Entertainment,Shogakukan-Shue...","Funimation,Geneon Entertainment USA",Madhouse,Action,"Adult Cast,Organized Crime",Seinen,https://cdn.myanimelist.net/images/anime/1906/...,https://www.youtube.com/watch?v=d4EbGC7fKnQ,"Apr 9, 2006 to Jun 25, 2006",Sundays at 02:35 (JST)
3,849,https://myanimelist.net/anime/849/Suzumiya_Har...,Suzumiya Haruhi no Yuuutsu,The Melancholy of Haruhi Suzumiya,涼宮ハルヒの憂鬱,Suzumiya Haruhi no Yuuutsu,TV,Light novel,14.0,Finished Airing,False,23 min per ep,PG-13 - Teens 13 or older,7.83,475447.0,888.0,179,909771,15321,If a survey were conducted to see if people be...,Suzumiya Haruhi no Yuuutsu aired in a nonlinea...,2006,spring,"Lantis,Kadokawa Shoten,Kadokawa Pictures Japan...","Funimation,Bandai Entertainment,Kadokawa Pictu...",Kyoto Animation,"Award Winning,Comedy,Mystery,Sci-Fi",School,Others,https://cdn.myanimelist.net/images/anime/1470/...,NaN,"Apr 3, 2006 to Jul 3, 2006",Mondays at 00:00 (JST)
4,934,https://myanimelist.net/anime/934/Higurashi_no...,Higurashi no Naku Koro ni,Higurashi: When They Cry,ひぐらしのなく頃に,"When the Cicadas Cry,The Moment the Cicadas Cry",TV,Visual novel,26.0,Finished Airing,False,24 min per ep,R - 17+ (violence & profanity),7.88,391899.0,809.0,229,797940,21247,Keiichi Maebara has just moved to the quiet li...,Geneon Entertainment USA initially licensed an...,2006,spring,"Geneon Universal Entertainment,Frontier Works,...","Sentai Filmworks,Geneon Entertainment USA",Studio Deen,"Horror,Mystery,Supernatural","Gore,Psychological",Others,https://cdn.myanimelist.net/images/anime/12/19...,NaN,"Apr 5, 2006 to Sep 27, 2006",Wednesdays at 01:30 (JST)


In [19]:
date_pattern = r'(\w{3} \d{1,2}, \d{4})'
animes['start_date'] = animes['aired_date'].str.extract(date_pattern)

animes['start_date'] = pd.to_datetime(animes['start_date'])

def get_season(month):
    if 4 <= month <= 6:
        return 'Spring'
    elif 7 <= month <= 9:
        return 'Summer'
    elif 10 <= month <= 12:
        return 'Fall'
    else:
        return 'Winter'

# Apply the function to create a new 'season' column
animes['season'] = animes['start_date'].dt.month.apply(get_season)

In [20]:
animes['Title'] = np.where(animes['title_english'].notnull(),
                           animes['title_english'],
                           animes['title']
                        )

In [21]:
idf = animes.interactive()

In [22]:
# Define Panel widgets
year_slider = pn.widgets.IntSlider(name='Year slider', start=2006, end=2022, step=1, value=2020)
year_slider

BokehModel(combine_events=True, render_bundle={'docs_json': {'78d18634-fff7-4d79-a41b-f55332228b2a': {'version…

In [23]:
choices = animes['type'].unique().tolist()
type_radio_button_group = pn.widgets.RadioButtonGroup(name='Select an option', options=choices, value=choices[0], button_type='primary')
type_radio_button_group

BokehModel(combine_events=True, render_bundle={'docs_json': {'ea6d11c2-f178-426f-9aa2-6624d0c4a23b': {'version…

In [24]:
choices = animes['genres'].str.split(',', expand=True).stack().str.strip().unique().tolist()
choices = sorted(choices)
genres_checkbox_group = pn.widgets.CheckBoxGroup(
    name='Checkbox Group', value=[], options=choices, inline=False)

genres_checkbox_group

BokehModel(combine_events=True, render_bundle={'docs_json': {'433ff771-db05-4d45-bfd2-f92e1aff9ad1': {'version…

In [25]:
choices = animes['season'].str.split(',', expand=True).stack().str.strip().unique().tolist()
season_checkbox_group = pn.widgets.CheckBoxGroup(
    name='Checkbox Group', value=['Winter'], options=choices, inline=False)

season_checkbox_group

BokehModel(combine_events=True, render_bundle={'docs_json': {'2bbdf1e3-bdf7-4968-ac43-f53fdc35da40': {'version…

In [26]:
choices = animes['demographics'].str.split(',', expand=True).stack().str.strip().unique().tolist()
demographics_checkbox_group = pn.widgets.CheckBoxGroup(
    name='Checkbox Group', value=choices, options=choices, inline=True)

demographics_checkbox_group

BokehModel(combine_events=True, render_bundle={'docs_json': {'59a61796-0f33-4700-8597-62ccfe247b21': {'version…

In [27]:
choices = animes['themes'].str.split(',', expand=True).stack().str.strip().unique().tolist()
themes_checkbox_group = pn.widgets.CheckBoxGroup(
    name='Checkbox Group', value=choices, options=choices, inline=False)

themes_checkbox_group

BokehModel(combine_events=True, render_bundle={'docs_json': {'85834c59-9af2-4a3c-afe3-62e7d3a70275': {'version…

In [53]:
animes[animes.index == 0]['genres'].str.split(',')[0]

['Comedy', 'Romance']

In [68]:
set(animes['genres'].str.split(',')[0]).issubset(set(['Comedy','Romance']))

True

In [86]:
# animes['genres'].str.split(',').apply(lambda x: set(x).issubset(set(['Comedy', 'Romance'])))

0         True
1        False
2        False
3        False
4        False
         ...  
16327    False
16328    False
16329    False
16330    False
16331    False
Name: genres, Length: 16332, dtype: bool

In [87]:
# is_include = idf['genres'].str.split(',').apply(lambda x: set(x).issubset(set(genres_checkbox_group.value)))
y = genres_checkbox_group.value
top_animes_across_year_pipeline = (
    idf[
        (idf['year'] == year_slider) &
        (idf['season'].isin(season_checkbox_group)) & 
        (idf['type'] == type_radio_button_group) &
        
        idf['genres'].str.split(',').apply(lambda x: set(x).issubset(set(y)))

        # (idf['demographics'].isin(demographics_checkbox_group))
        # (idf['themes'].isin(themes_checkbox_group))
    ]
    .sort_values(by='score', ascending=False)
    [['Title', 'score']]
    [:10]
    .reset_index(drop=True)
)


# top_animes_across_year_pipeline

In [29]:
top_animes_table = top_animes_across_year_pipeline.pipe(
    pn.widgets.Tabulator,
    pagination='remote', page_size = 10, sizing_mode='stretch_width'
)

In [30]:
template = pn.template.FastListTemplate(
    title='Top Animes',
    sidebar=[
        year_slider,
        season_checkbox_group,
        type_radio_button_group,
        demographics_checkbox_group,
        genres_checkbox_group,
        themes_checkbox_group,
    ],
    main=[
        pn.Column(top_animes_table.panel(width=700))
    ],
    accent_base_color="#88d8b0",
    header_background="#88d8b0",
)

_ = template.servable()